In [1]:
import numpy as np
import pandas as pd
import random
import datetime
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
features = pd.read_csv("features.csv")
output = pd.read_csv("output.csv")

In [3]:
#features.columns

In [4]:
#features = features[['temperature', 'humidity', 'precipitation', 'windspeed', 'age', 'weight', 'height', 'sex', 'fatpercentage', 'bmi']]

In [5]:
features.tail()

,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi,cold_resistance,scoring_difference,adjusted_cold_resistance,preference_factor
99995,10,8,94,18,38,48.0,131.0,1,36.904478,27.970398,100.0,0.000000,98.00,98
99996,17,85,77,6,45,76.0,173.0,1,35.422117,25.393431,80.0,0.000000,76.80,96
99997,21,78,36,40,31,48.0,174.0,1,20.754970,15.854142,77.5,0.000000,79.05,102
99998,-15,42,89,34,46,38.0,113.0,0,30.091489,29.759574,90.0,-6.033333,92.70,103
99999,5,88,78,39,37,95.0,134.0,1,66.598528,52.907106,100.0,-3.666667,97.00,97


In [6]:
output.tail()

,thermal,hoodie,fleece,wool,light_down,thick_down,wind_breaker,umbrella,winter_boots,user_feedback
99995,True,True,True,False,True,True,False,False,True,1
99996,True,True,True,True,True,False,False,True,False,1
99997,True,True,True,True,False,True,True,True,False,1
99998,False,False,False,False,True,True,False,False,True,0
99999,False,False,True,False,True,False,True,False,False,1


In [7]:
output.columns

Index(['thermal', 'hoodie', 'fleece', 'wool', 'light_down', 'thick_down',
       'wind_breaker', 'umbrella', 'winter_boots', 'user_feedback'],
      dtype='object')

In [8]:
output = output[['thermal', 'hoodie', 'fleece', 'wool', 'light_down', 'thick_down','wind_breaker', 'umbrella', 'winter_boots']]
for i in output.columns:
    output[i] = output[i].astype(int)

In [9]:
output.tail()

,thermal,hoodie,fleece,wool,light_down,thick_down,wind_breaker,umbrella,winter_boots
99995,1,1,1,0,1,1,0,0,1
99996,1,1,1,1,1,0,0,1,0
99997,1,1,1,1,0,1,1,1,0
99998,0,0,0,0,1,1,0,0,1
99999,0,0,1,0,1,0,1,0,0


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(features, output[['thermal']], test_size=0.25, random_state=10)

In [11]:
#https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7
D_train = xgb.DMatrix(X_train, label = Y_train)
D_test = xgb.DMatrix(X_test, label = Y_test)

In [18]:
#best params so far based on grid search
param = {
    'eta': 0.3, 
    'max_depth': 8,  
    'objective': 'multi:softprob',  
    'num_class': 3,
    'gamma' : 0.4,
    'colsample_bytree': 0.7,
    'min_child_weight': 1
    } 

steps = 20  # The number of training iterations

In [19]:
model = xgb.train(param, D_train, steps)

[21:40:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [24]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
preds = model.predict(D_test, ntree_limit=model.best_ntree_limit)


#best_preds = np.asarray([np.argmax(line) for line in preds])


print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9135661962758537
Recall = 0.8651391973394335
Accuracy = 0.91428


C:\Users\User\anaconda3\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [25]:
print(preds)

[[0.98806936 0.00970558 0.00222513]
 [0.41207093 0.5762061  0.01172303]
 [0.88238597 0.11061575 0.00699827]
 ...
 [0.16319945 0.826028   0.01077263]
 [0.7883162  0.20212494 0.00955886]
 [0.9803038  0.01681835 0.00287786]]
